# Rent

In [ ]:
import folium
import geopandas as gpd
import pandas as pd

In [ ]:
cd /home/lisa/work/people_places_germany/code

In [ ]:
from spatial_autocorrelation import lisa

In [ ]:
clusters = gpd.read_parquet(
    "/data/cluster_data/clusters_umap_freiburg_100_3_gaussian_euclidean_ward_2.pq"
).reset_index()

In [ ]:
# clusters.explore(column=clusters.index,cmap='tab20')

In [ ]:
gdf = gpd.read_parquet("/data/processed_data/rent_2022.parquet").reset_index()
data = gpd.read_parquet("/data/processed_data/rent_2022.parquet").reset_index()
gdf = gdf.cx[4150000:4170000, 2760000:2780000]

In [ ]:
gdf

# Raw Data

In [ ]:
# gdf.explore("durchschnMieteQM", cmap="coolwarm", prefer_canvas=True)

# Local Spatial Autocorrelation

In [ ]:
mi, gdf_05 = lisa(gdf, "durchschnMieteQM", 0.05)

In [ ]:
print(f"Moran's I: {mi.I}, p-value: {mi.p_sim}")

In [ ]:
# gdf_05.explore("cluster", prefer_canvas=True, cmap=["#d7191c","#fdae61","#abd9e9","#2c7bb6","lightgrey"])

# Cluster Aggregation

In [ ]:
data = data.rename(columns={"GITTER_ID_100m": "ID"})
cells = data[["ID", "geometry"]]

overlap = gpd.overlay(cells, clusters, how="intersection")
overlap["area"] = overlap.geometry.area
largest_overlap = overlap.loc[overlap.groupby(overlap["ID"])["area"].idxmax()]
data["cluster"] = data.merge(largest_overlap[["ID", "label"]], how="left", on="ID")[
    "label"
]

d = []
for i in range(0, int(data["cluster"].max())):
    d.append(
        {
            "Average rent": data[data["cluster"] == i]["durchschnMieteQM"].mean(),
            "Average rent count": data[data["cluster"] == i][
                "durchschnMieteQM"
            ].count(),
        }
    )

rent_stats = pd.DataFrame(d)

In [ ]:
data.to_parquet(
    "/home/lisa/work/people_places_germany/Notebooks/temp_data/rent.parquet"
)

In [ ]:
rent_stats["label"] = rent_stats.index
rent_stats = clusters.merge(rent_stats, on="label")

In [ ]:
m = rent_stats.explore(
    column="Average rent",
    cmap="coolwarm",
    prefer_canvas=True,
    name="cluster aggregation",
)

gdf_05.explore(
    "sig_cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    name="local spatial autocorrelation",
    m=m,
)

gdf.explore(
    "durchschnMieteQM", cmap="coolwarm", prefer_canvas=True, name="raw data", m=m
)

# Add a layer control to toggle the layers on and off
folium.LayerControl().add_to(m)

# Display the map
m